# AFAP Trend Engine Validation

This notebook validates the **Trend Engine** in AFAP Phase 3 (Locked Engines).

The Trend Engine evaluates **directional financial trends** across multiple years
using locked ratio outputs produced by the Ratio Engine.

This notebook verifies that:
1. Ratio outputs are structurally compatible with the Trend Engine
2. Metrics are correctly flattened for trend computation
3. Trend signals are computed accurately
4. Engine outputs conform to AFAP schema requirements
5. Outputs are ready for orchestration and AI interpretation


## 1. Environment Setup

We first configure the Python environment and import AFAP engines.
The project root is added to `sys.path` to allow clean engine imports.


In [1]:
import sys
from pathlib import Path

# Define project root
project_root = Path.cwd().parent
sys.path.append(str(project_root))

# Import the Trend Engine
from engines.trend_engine import trend_engine

# Check engines package path
import engines
print("Engines package path:", engines.__file__)


Engines package path: c:\Users\ADMIN\Documents\My Documents\MyDataAnalysis\Financial statement analysis\financial-analysis-pipeline\engines\__init__.py


---

## 2️⃣ Load Financial Data

We load cleaned financial statements and inspect the first few rows to ensure data integrity.


In [2]:
import pandas as pd

financials = pd.read_csv("../data/cleaned/financial_statements.csv")
financials.head()


,Company,Year,FS Category,FS Subcategory,Statement,Amount
0,Acme Manufacturing Ltd,2020,Assets,Current Assets,Balance Sheet,3109667
1,Acme Manufacturing Ltd,2020,Assets,Non-Current Assets,Balance Sheet,905812
2,Acme Manufacturing Ltd,2020,Equity,Equity,Balance Sheet,2152630
3,Acme Manufacturing Ltd,2020,Expenses,COGS,Income Statement,373114
4,Acme Manufacturing Ltd,2020,Expenses,Finance Costs,Income Statement,2929304


---

## 3️⃣ Compute Base Ratios

The Trend Engine depends on locked ratio outputs from `ratio_engine_core`.  
Here, we compute these ratios and convert them to a DataFrame for trend analysis.


In [3]:
from engines.ratio_engine_core import ratio_engine

# Generate core ratios
ratios_core = ratio_engine(financials)

# Convert to DataFrame
ratios_df = pd.DataFrame(ratios_core)

# Sort and reset index
ratios_df.sort_values(["Company", "Year"], inplace=True)
ratios_df.reset_index(drop=True, inplace=True)

ratios_df.head()


✅ ratio_engine output validated successfully.


,engine,Company,Year,metrics,flags,severity,explanation
0,ratio_engine,Acme Manufacturing Ltd,2020,"{'current_ratio': 1.5876421245028514, 'quick_r...",{},stable,Canonical financial ratios
1,ratio_engine,Acme Manufacturing Ltd,2021,"{'current_ratio': 3.0794864428132476, 'quick_r...",{},stable,Canonical financial ratios
2,ratio_engine,Acme Manufacturing Ltd,2022,"{'current_ratio': 3.4109333800409956, 'quick_r...",{},stable,Canonical financial ratios
3,ratio_engine,Acme Manufacturing Ltd,2023,"{'current_ratio': 4.67288599769455, 'quick_rat...",{},stable,Canonical financial ratios
4,ratio_engine,Banyan Retail Co,2020,"{'current_ratio': 2.513537113097586, 'quick_ra...",{},stable,Canonical financial ratios


---

## 4️⃣ Trend Engine Function Signature

The Trend Engine evaluates **directional trends** (YoY or multi-year) for the following ratios:

- Current Ratio
- Gross Margin
- Net Margin
- Asset Turnover
- Debt-to-Equity
- Return on Equity (ROE)

It outputs a list of dictionaries with:

- `engine`: Engine name
- `Company`: Company name
- `Year`: Most recent year
- `metrics`: Trend value, start/end year, and ratio
- `flags`: Indicators (e.g., deteriorating_trend)
- `severity`: Risk level ("watch" for deterioration)
- `explanation`: Human-readable explanation

> **Note:** Baseline years (first available year per company) cannot have a trend and are skipped.


## 5️⃣ Flatten Ratio Engine Output for Trend Analysis

The Trend Engine operates on **time-series ratio values**.  
However, the Ratio Engine outputs ratios inside a nested `metrics` dictionary.

To enable trend computation, we **flatten** the `metrics` field into explicit columns.

This transformation:
- Preserves engine integrity (no engine modification)
- Produces a time-series friendly structure
- Acts as a **validation-only preparation step**


In [4]:
# Extract metrics dict into columns
metrics_df = pd.json_normalize(ratios_df['metrics'])
ratios_flat = pd.concat([ratios_df[['Company', 'Year']], metrics_df], axis=1)

ratios_flat.head()


,Company,Year,current_ratio,quick_ratio,gross_margin,operating_margin,net_margin,debt_equity,interest_coverage,asset_turnover,roa,roe
0,Acme Manufacturing Ltd,2020,1.587642,1.587642,0.867371,0.289107,-1.546148,0.982524,0.277650,0.700595,-1.083224,-2.020627
1,Acme Manufacturing Ltd,2021,3.079486,3.079486,0.854884,0.212159,-0.787051,1.358428,0.388825,0.772164,-0.607732,-2.132176
2,Acme Manufacturing Ltd,2022,3.410933,3.410933,0.968294,0.680335,0.301875,0.696958,2.446630,1.990334,0.600832,1.548439
3,Acme Manufacturing Ltd,2023,4.672886,4.672886,0.771416,0.510596,0.182190,1.476542,2.003211,1.965558,0.358106,1.137251
4,Banyan Retail Co,2020,2.513537,2.513537,0.757082,0.632036,0.180853,1.047567,2.023443,3.234804,0.585025,0.796404


### ✅ Resulting Structure

The flattened DataFrame now contains:

- `Company`
- `Year`
- Individual ratio columns:
  - `current_ratio`
  - `gross_margin`
  - `net_margin`
  - `asset_turnover`
  - `debt_equity`
  - `roe`

This structure matches the **Trend Engine’s expected input contract**.


## 6️⃣ Execute Trend Engine

We now pass the flattened ratio DataFrame into the locked Trend Engine.

The Trend Engine:
- Groups data by company
- Sorts by year
- Computes directional change per ratio
- Anchors results to the most recent year
- Flags deteriorating trends


In [5]:
trend_results = trend_engine(ratios_flat)
trend_results[:2]


✅ trend_engine output validated successfully.


[{'engine': 'trend_engine',
  'Company': 'Acme Manufacturing Ltd',
  'Year': 2023,
  'metrics': {'ratio': 'current_ratio',
   'trend_value': np.float64(3.0852438731916987),
   'from_year': 2020,
   'to_year': 2023},
  'flags': {'deteriorating_trend': np.False_},
  'severity': 'stable',
  'explanation': 'current_ratio trend stable or improving.'},
 {'engine': 'trend_engine',
  'Company': 'Acme Manufacturing Ltd',
  'Year': 2023,
  'metrics': {'ratio': 'gross_margin',
   'trend_value': np.float64(-0.09595568047712277),
   'from_year': 2020,
   'to_year': 2023},
  'flags': {'deteriorating_trend': np.True_},
  'severity': 'watch',
  'explanation': 'Negative trend observed in gross_margin.'}]

### 📤 Example Output Interpretation

Each Trend Engine record represents **one ratio trend per company**.

Example:

- **Company:** Acme Manufacturing Ltd
- **Ratio:** current_ratio
- **Period:** 2020 → 2023
- **Trend Value:** +3.08
- **Severity:** stable

Another record shows:

- **Ratio:** gross_margin
- **Trend Value:** −0.095
- **Flag:** deteriorating_trend = True
- **Severity:** watch

This demonstrates:
- Independent evaluation per ratio
- Directional (not absolute) risk signaling
- Human-readable explanations embedded in output


## 7️⃣ Structural Validation Checks

We now confirm that Trend Engine outputs meet AFAP Phase 3 schema expectations.


In [6]:
# Ensure correct engine labeling
assert all(r["engine"] == "trend_engine" for r in trend_results)

# Ensure required keys exist
required_keys = {"engine", "Company", "Year", "metrics", "flags", "severity", "explanation"}
assert all(required_keys.issubset(r.keys()) for r in trend_results)

# Ensure trend_value is numeric
assert all(
    isinstance(r["metrics"]["trend_value"], (int, float))
    for r in trend_results
)

print("✅ Trend Engine structural validation passed.")


✅ Trend Engine structural validation passed.


## 8️⃣ Design Notes (Phase 3 Compliance)

- Trend Engine does **not** interpret financial meaning beyond direction
- No thresholds or business logic leakage
- No AI reasoning embedded
- Baseline years are intentionally skipped
- Output is deterministic and replayable

This preserves:
- Audit traceability
- Engine composability
- Downstream AI neutrality


## 9️⃣ Orchestration Readiness

Trend Engine outputs are now ready for:

- Aggregation by Orchestrator
- Risk layering with other engines
- AI interpretation at the **platform layer**
- UI rendering without recalculation

Trend signals are **inputs**, not conclusions.


## ✅ Validation Summary

- Ratio outputs successfully flattened
- Trend Engine executed as designed
- Directional signals correctly detected
- Deteriorating trends flagged
- Outputs conform to AFAP Phase 3 schema

Trend Engine validation complete.
